In [ ]:
# ШАГ 1: Установка обновленного фреймворка
!pip install --no-cache-dir git+https://github.com/St0rmMaster/binance_data_framework.git --quiet

# ШАГ 2: Минимально необходимые импорты
from google.colab import output
from binance_data_framework import BinanceUSClient, GoogleDriveDataManager, DataDownloaderUI

# ШАГ 3: Инициализация API клиента
api_client = BinanceUSClient()

# ШАГ 4: Инициализация менеджера данных
# Используется только GoogleDriveDataManager, без параметров

# ШАГ 5: Включение поддержки виджетов
output.enable_custom_widget_manager()

# ШАГ 6: Создание и отображение интерфейса
ui = DataDownloaderUI(api_client=api_client, db_manager=GoogleDriveDataManager())
ui.display()